In [124]:
!sudo apt get-update
!sudo apt upgrade

E: Invalid operation get-update
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Calculating upgrade... Done
The following packages have been kept back:
  libcudnn9-cuda-12 libcudnn9-dev-cuda-12 libnccl-dev libnccl2
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.


In [125]:
import torch

# Example 1

In [126]:
x = torch.tensor(3.0, requires_grad=True)
print(x)

tensor(3., requires_grad=True)


In [127]:
y = x**2
print(y)

tensor(9., grad_fn=<PowBackward0>)


In [128]:
y.backward()

In [129]:
x.grad

tensor(6.)

# Example 2

In [130]:
x = torch.tensor(4.0, requires_grad=True)
y = x**2
z = torch.sin(y)

print(x)
print(y)
print(z)

tensor(4., requires_grad=True)
tensor(16., grad_fn=<PowBackward0>)
tensor(-0.2879, grad_fn=<SinBackward0>)


In [131]:
z.backward()

In [132]:
x.grad

tensor(-7.6613)

# Example 3

# 1st Method

In [133]:
# Inputs
x = torch.tensor(6.7, requires_grad=True)
y = torch.tensor(0.0, requires_grad=True)

# Weights and biases
w = torch.tensor(1.0)
b = torch.tensor(0.0)

In [134]:
# Define Cross Entropy Binary Loss for scaler
def binary_cross_entropty(prediction, target):
  epsilon = 1e-8 # To prevent lof(0)
  prediction = torch.clamp(prediction, epsilon, 1-epsilon)
  return -(target - torch.log(prediction) + (1 - target) * torch.log(1 - prediction))

In [135]:
# Forward propagation
z = w*x + b # wigthed Sum
y_pred = torch.sigmoid(z) # predicted probabilty
loss = binary_cross_entropty(y_pred, y) # computing cross entropy binar loss

In [136]:
print(z)
print(y_pred)
print(loss)

tensor(6.7000, grad_fn=<AddBackward0>)
tensor(0.9988, grad_fn=<SigmoidBackward0>)
tensor(6.6999, grad_fn=<NegBackward0>)


In [137]:
# Derivatives
# dloss/dy_pred : Loss with respect to prediction
dloss_dy_pred = (y_pred - y)/(y_pred * (1 -  y_pred))
# dy_pred/dz : Prediction with respect to z (sigmoid derivative)
dy_pred_dz = y_pred * (1 -  y_pred)
# dz/dw and dz/db : z  with respect to w and b
dz_dw = x # dz/dw = x
dz_db = 1.0 # bias contricutes directly to prediction

dL_dw = dloss_dy_pred * dy_pred_dz * dz_dw
dL_db = dloss_dy_pred * dy_pred_dz * dz_db

In [138]:
print(f"Manual gradient of loss w.r.t w: {dL_dw}")
print(f"Manual gradient of loss w.r.t b: {dL_db}")

Manual gradient of loss w.r.t w: 6.691762447357178
Manual gradient of loss w.r.t b: 0.998770534992218


# 2nd Method

In [139]:
x = torch.tensor(6.7)
y = torch.tensor(0.0)

In [140]:
w = torch.tensor(1.0, requires_grad=True)
b = torch.tensor(0.0, requires_grad=True)

In [141]:
z = w*x + b
y_pred = torch.sigmoid(z)
loss = binary_cross_entropty(y_pred, y)

In [142]:
loss.backward()

In [143]:
print(w.grad)
print(b.grad)

tensor(6.7000)
tensor(1.)


# Another example

In [144]:
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)

In [145]:
y = (x**2).mean()
print(y)

tensor(4.6667, grad_fn=<MeanBackward0>)


In [146]:
y.backward()

In [147]:
x.grad

tensor([0.6667, 1.3333, 2.0000])

In [148]:
x = torch.tensor(2.0, requires_grad=True)

In [149]:
y = x**2

In [150]:
y.backward()

In [151]:
x.grad

tensor(4.)

In [152]:
# Do this to clear up the gradients
x.grad.zero_()

tensor(0.)

# How to disable gradient tracking

In [153]:
x = torch.tensor(2.0, requires_grad=True)

In [154]:
y = x**2
print(y)

tensor(4., grad_fn=<PowBackward0>)


In [155]:
y.backward()

In [156]:
x.grad

tensor(4.)

In [157]:
# option 1: requires_grad_(False)
# option 2: detach()
# option 3: torch.no_grad()

Option 1:

In [158]:
x.requires_grad_(False)
print(x)

tensor(2.)


In [159]:
y = x**2
print(y)

tensor(4.)


In [160]:
y.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

Option 2:

In [161]:
x = torch.tensor(2.0, requires_grad=True)
z = x.detach()
print(x)
print(z)

tensor(2., requires_grad=True)
tensor(2.)


In [162]:
y1 = z**2
print(y1)

tensor(4.)


In [163]:
y1.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

Option 3:

In [164]:
x = torch.tensor(2.0, requires_grad=True)
print(x)

tensor(2., requires_grad=True)


In [165]:
with torch.no_grad():
  y = x**2
  print(y)

tensor(4.)


In [166]:
y.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn